In [0]:
import pandas as pd
import numpy as np
import os
from lxml import etree
from tqdm import tqdm
import re
import pickle
from uuid import uuid4
import shelve
pd.set_option('display.max_columns', None)

## Parsing Copyright Registrations

### A few considerations:

1. Copyright registrations that happened after 1963 were renewed automatically, so there's no point in checking them for an explicit renewal.

2. "Every registration should have a registration number, such as A56505, but these are not unique. Numbering was restarted in 'Third Series' (1947–) so there is quite a bit of overlap between this and the 'New Series.' For example, **a registration number and date is always required to distinguish 'A56505/1951-06-06' from 'A56505/1932-10-12'**."

3. "The *duplicateOf* attribute indicates that the entry with the attribute contains the same information as the entry it points to and adds nothing to it. They must both have identical registration numbers and dates. \[\.\.\.\] All other things being equal, later duplicates should refer to earlier entries. If there are multiple duplications all duplicates should point to the same "master" registration. That entry must not have a *duplicateOf* attribute. So, **when processing, any entries carrying a *duplicateOf* attribute can be skipped, since -- if an entry is truly a duplicate -- it adds nothing the copyright history of the work.** When importing into a database, this will assure that there is only one row with the registration number/date combination. **Renewals should be linked to registrations without the *duplicateOf* attribute.**

4. Some of the registration records are foreign/interim registrations (AF and AI), which aren't subject to the normal renewal rules. These should be removed.

In [0]:
# load up directory of copyright registrations coded in XML

base_path = os.getcwd()

registration_files = os.path.join(base_path, 'catalog_of_copyright_entries_project-master', 'xml')

reg_directories = [x for x in os.listdir(registration_files)]

xml_data = []
for directory in tqdm(reg_directories):
    # Copyright registrations that happened after 1963 were renewed automatically,
    # so there's no point in checking them for an explicit renewal.
    if int(directory) > 1963:
        pass
    else:
        files = [x for x in os.listdir(os.path.join(registration_files, directory)) if x[-4:] == '.xml']
        for file in files:
            tree = etree.parse(os.path.join(registration_files, directory, file))
            root = tree.getroot()
            entries = root.findall('copyrightEntry')
            for entry in entries:
                # any entries carrying a *duplicateOf* attribute can be skipped
                if 'duplicateOf' in entry.attrib:
                    pass
                else:
                    try:
                        regnum = entry.attrib['regnum']
                    except:
                        regnum = 'ERROR'
                    try:
                        ident = entry.attrib['id']
                    except:
                        ident = 'ERROR'
                    temp = {'control_number': regnum, 'id': ident, 'xml_year': directory}
                    for child in entry:
                        if child.tag == 'regDate':
                            try:
                                temp['control_date'] = child.attrib['date']
                                temp['regDate'] = child.text
                            except KeyError:
                                temp['control_date'] = 'ERROR'
                                temp['regDate'] = child.text
                        elif child.tag == 'regdate':
                            try:
                                temp['control_date'] = child.attrib['date']
                                temp['regDate'] = child.text
                            except KeyError:
                                temp['control_date'] = 'ERROR'
                                temp['regDate'] = child.text
                        else:
                            temp[child.tag] = child.text
                    xml_data.append(temp)

100%|██████████| 46/46 [00:15<00:00,  2.94it/s]


In [0]:
# create dataframe (DF) of parsed registrations
registrations_df = pd.DataFrame(xml_data).set_index('id')
len(registrations_df)

711409

In [0]:
# how many are international/interim copyrights?
len(registrations_df[(registrations_df['control_number'].str.contains('AI', case=True, regex=False)) |
                     (registrations_df['control_number'].str.contains('AF', case=True, regex=False))])

131148

In [0]:
# remove international/interim registrations
registrations_df = registrations_df[registrations_df['control_number'].str.contains('AI', case=True, regex=False) == False]
registrations_df = registrations_df[registrations_df['control_number'].str.contains('AF', case=True, regex=False) == False]
len(registrations_df)

580261

In [0]:
# are any of the registrations missing a control date?
len(registrations_df[registrations_df['control_date'] == 'ERROR'])

1

In [0]:
registrations_df[registrations_df['control_date'] == 'ERROR']

,control_number,xml_year,author,title,edition,publisher,desc,control_date,regDate,copies,copyDate,affDate,regNum,lccn,note,newMatterClaimed,series,vol,volumes,claimant,prevPub,additionalEntry,prev-regNum,noticeDate,role
id,,,,,,,,,,,,,,,,,,,,,,,,,
5A28A65B-6E24-1014-A696-AD35B3FDE14F,A6151,1946,"author,",Mine own executioner,NaN,None,"xiv p., 1 l., 336 p. 21½ͨͫ.",ERROR,10Sep46,2c,29Aug46,NaN,A6151,46-11904,"""First printed in the United States, 1946."" I....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prev. reg. AI 29232,NaN,NaN


In [0]:
# fill in that date from regDate
registrations_df.at['5A28A65B-6E24-1014-A696-AD35B3FDE14F', 'control_date'] = '1946-09-10'

In [0]:
# are any of the registrations missing a control number?
len(registrations_df[(registrations_df['control_number'] == 'ERROR')])

158

In [0]:
# check to see if missing control numbers equals the same number in the regNum field
len(registrations_df[(registrations_df['control_number'] == 'ERROR') & (registrations_df['regNum'].isnull())])

157

In [0]:
# apparently there's one outlier:
registrations_df[(registrations_df['control_number'] == 'ERROR') & (registrations_df['regNum'].notnull())]

,control_number,xml_year,author,title,edition,publisher,desc,control_date,regDate,copies,copyDate,affDate,regNum,lccn,note,newMatterClaimed,series,vol,volumes,claimant,prevPub,additionalEntry,prev-regNum,noticeDate,role
id,,,,,,,,,,,,,,,,,,,,,,,,,
0BFA5A57-79D5-1014-8221-AE20959418B2,ERROR,1948,None,"J. E. Klingberg, adventurer in faith",NaN,None,200 p. illus.,1948-12-15,15Dec48,NaN,NaN,NaN,A28518,NaN,a biography,NaN,[Baptist General Conference of America] Centen...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# fill in that control number from regNum
registrations_df.at['0BFA5A57-79D5-1014-8221-AE20959418B2', 'control_number'] = 'A28518'

In [0]:
# what about control numbers that, uh, aren't control numbers?
(registrations_df.sort_values(by=['control_number'], ascending=False)).head(15)

,control_number,xml_year,author,title,edition,publisher,desc,control_date,regDate,copies,copyDate,affDate,regNum,lccn,note,newMatterClaimed,series,vol,volumes,claimant,prevPub,additionalEntry,prev-regNum,noticeDate,role
id,,,,,,,,,,,,,,,,,,,,,,,,,
7EE3500E-6E18-1014-9400-B664922230AB,no. 8,1942,NaN,Picture post.,NaN,pubd.,NaN,1942-02-21,"Feb. 21, 1942",1 c,"Mar. 16, 1942",NaN,A ad int. 27152,NaN,NaN,NaN,NaN,v. 14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7F934843-6E18-1014-9400-B664922230AB,no. 5288,1942,NaN,Punch,NaN,pubd.,NaN,1942-07-01,"July 1, 1942",1 c,Aug. 26,NaN,A ad int. 27265,NaN,NaN,NaN,NaN,v. 202,NaN,NaN,NaN,"No. 5297,",NaN,NaN,NaN
7D944988-6E18-1014-9400-B664922230AB,no. 5261,1942,NaN,Punch.,NaN,pubd.,NaN,1941-12-31,"Dec. 31, 1941",1 c,Feb. 25,NaN,A ad int. 27006,NaN,NaN,NaN,NaN,v. 201,NaN,NaN,NaN,,NaN,NaN,NaN
7F2608C3-6E18-1014-9400-B664922230AB,no. 4,1942,NaN,Lilliput,NaN,pubd.,NaN,1941-10,"Oct., 1941",1 c,"Oct. 8, 1941",NaN,A ad int. 27179,NaN,NaN,NaN,NaN,v. 9,NaN,NaN,NaN,,NaN,NaN,NaN
7F3396D2-6E18-1014-9400-B664922230AB,no. 4,1942,NaN,Picture post.,NaN,pubd.,NaN,1941-10-25,"Oct 25, 1941",1 c,"Nov. 24, 1941",NaN,A ad int. 27185,NaN,NaN,NaN,NaN,v. 13,NaN,NaN,NaN,"No. 13,",NaN,NaN,NaN
7F0C9288-6E18-1014-9400-B664922230AB,no. 172,1942,NaN,Electronic engineering.,NaN,pubd.,NaN,1942-06,"June, 1942",1 c,"June 23, 1942",NaN,A ad int. 27191,NaN,NaN,NaN,NaN,v. 16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7D90C472-6E18-1014-9400-B664922230AB,no. 12,1942,NaN,Picture post.,NaN,pubd.,NaN,1941-09-20,"Sept. 20, 1941",1 c,"Oct. 3, 1941",NaN,A ad int. 27027,NaN,NaN,NaN,NaN,v. 12,NaN,NaN,NaN,"No. 7,",NaN,NaN,NaN
9B0B664D-6D0F-1014-9803-8BC52073E431,l4Aug59 A404569,1959,None,Clinical coordination of anatomy and physiology,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,l4Aug59 A404569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74C5BB13-6E97-1014-9357-E9A0C844EAF1,c1952,1952,None,"How to grow minnows. Ft. Worth, Tex.",NaN,None,63 p. illus.,1952-09-09,9Sep52,NaN,NaN,NaN,AA223224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# we should fix some of these
registrations_df.at['7EE3500E-6E18-1014-9400-B664922230AB', 'control_number'] = 'A ad int. 27152'
registrations_df.at['7F934843-6E18-1014-9400-B664922230AB', 'control_number'] = 'A ad int. 27265'
registrations_df.at['7D944988-6E18-1014-9400-B664922230AB', 'control_number'] = 'A ad int. 27006'
registrations_df.at['7F2608C3-6E18-1014-9400-B664922230AB', 'control_number'] = 'A ad int. 27179'
registrations_df.at['7F3396D2-6E18-1014-9400-B664922230AB', 'control_number'] = 'A ad int. 27185'
registrations_df.at['7F0C9288-6E18-1014-9400-B664922230AB', 'control_number'] = 'A ad int. 27191'
registrations_df.at['7D90C472-6E18-1014-9400-B664922230AB', 'control_number'] = 'A ad int. 27027'
registrations_df.at['9B0B664D-6D0F-1014-9803-8BC52073E431', 'control_number'] = 'A404569'
registrations_df.at['74C5BB13-6E97-1014-9357-E9A0C844EAF1', 'control_number'] = 'A223224'
registrations_df.at['65934EB9-6E24-1014-A696-AD35B3FDE14F', 'control_number'] = 'A3347'
registrations_df.at['D0F4FB38-6D0A-1014-853A-B63123D40E62', 'control_number'] = 'A17092'
registrations_df.at['F589B7BA-6D13-1014-B63C-9736EB68D6D5', 'control_number'] = 'A602007'
registrations_df.at['05E3281B-6D14-1014-B63C-9736EB68D6D5', 'control_number'] = 'A601644'

In [0]:
# most registration number seem to hover around 7 characters, let's see if this is standard
len(registrations_df[registrations_df['control_number'].str.len() > 7])

18120

In [0]:
# huh. well, let's look at some of these
registrations_df[registrations_df['control_number'].str.len() > 7].head(5)

,control_number,xml_year,author,title,edition,publisher,desc,control_date,regDate,copies,copyDate,affDate,regNum,lccn,note,newMatterClaimed,series,vol,volumes,claimant,prevPub,additionalEntry,prev-regNum,noticeDate,role
id,,,,,,,,,,,,,,,,,,,,,,,,,
85D6B068-767F-1014-AA44-F9F4E24D38B2,A765572 A765573,1924,None,Physiology and health,NaN,None,2 v. illus. 19ͨͫ,1924-01-02,"Jan. 2, 1924",2c.,NaN,Jan. 3,"A 765572, 765573",24-1323,NaN,additional subject matter,His Physiology and health series,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85E17D36-767F-1014-AA44-F9F4E24D38B2,A765638 A765639 A765640 A765954 A765955,1924,None,Principles and practice of telephony,1st ed.,None,"5 v. illus., diagrs. 19½ͨͫ",1923-12-29,"Dec. 29, 1923",2c.,NaN,"Feb. 7, 1924","A 765638-765640, 765954, 765955",24-1728,CONTENTS.—[v. 1] Principles and apparatus.—[v....,NaN,NaN,v. 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86CA40D2-767F-1014-AA44-F9F4E24D38B2,A765986 A765987 A765988 A765989 A765990 A76599...,1924,None,Arkansas reports,NaN,None,13 v. 23½ͨͫ,1923-11-19,Nov. 19,2c.,"Feb. 11, 1924",Feb. 7,A 765986-765998,NaN,"cases determined … January, 1921, to July, 1923",NaN,NaN,v. 159,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86E0E4ED-767F-1014-AA44-F9F4E24D38B2,A777076 A777077 A777078 A777079 A777080 A77708...,1924,None,"Texas and Southwestern reporter digest, includ...",NaN,None,2 v. 26ͨͫ,1923-11-15,Nov. 15,2c.,"Feb. 11, 1924",Feb. 7,A 777076-777091,NaN,CONTENTS.—v. 2. Appeal and error*-Assignment o...,NaN,NaN,v. 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876E3661-767F-1014-AA44-F9F4E24D38B2,A777015 A777016,1924,NaN,Library of railway practice,NaN,None,"2 v. fronts., illus., diagrs. 21½ͨͫ",1924-01-15,"Jan. 15, 1924",2c.,Jan. 26,Feb. 8,"A 777015, 777016",24-4029,a general reference work on locomotive science...,introductory chapters by Hugh K. Christie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# crap crap crap
registrations_df['control_number'].str.contains(' ').sum()

4414

In [0]:
len(registrations_df[registrations_df['control_number'].str.len() > 15])

1751

In [0]:
len(registrations_df[registrations_df['control_number'].str.len() > 100])

84

In [0]:
len(registrations_df[registrations_df['control_number'].str.len() > 350])

3

In [0]:
registrations_df[registrations_df['control_number'].str.len() > 350]

,control_number,xml_year,author,title,edition,publisher,desc,control_date,regDate,copies,copyDate,affDate,regNum,lccn,note,newMatterClaimed,series,vol,volumes,claimant,prevPub,additionalEntry,prev-regNum,noticeDate,role
id,,,,,,,,,,,,,,,,,,,,,,,,,
1A57A4C0-7289-1014-A093-F050B7410E2F,A144 A16 A17 A18 A19 A20 A21 A22 A23 A24 A25 A...,1954,None,Some notes on autoclave curing for Portland ce...,NaN,None,5 l.,1954-05-20,20May54,NaN,NaN,NaN,A144-015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CC8FECD3-6D17-1014-B416-88728C01EDA4,BB19768 BB19769 BB19770 BB19771 BB19772 BB1977...,1962,None,Out of the past. Installment 1-49. Illustrator...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,BB19768 BB19769 BB19770 BB19771 BB19772 BB1977...,NaN,in notice,NaN,"(In Daily news, New York, Nov. 19, 1961-Jan. 6...",NaN,NaN,NaN,NaN,NaN,NaN,1961,NaN
B9967899-6D13-1014-B63C-9736EB68D6D5,BB21954 BB21955 BB21956 BB21957 BB21958 BB2195...,1962,None,The china governess. Installment 1-49. By Marg...,NaN,None,NaN,1962-09-27,9Sep-27Oct62,NaN,NaN,NaN,BB21954 BB21955 BB21956 BB21957 BB21958 BB2195...,NaN,NaN,NaN,"(In Daily news, New York, Sept. 9- Oct. 27, 19...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# id number 1A57A4C0-7289-1014-A093-F050B7410E2F provides an intersting example...
registrations_df.at['1A57A4C0-7289-1014-A093-F050B7410E2F', 'control_number']

'A144 A16 A17 A18 A19 A20 A21 A22 A23 A24 A25 A26 A27 A28 A29 A30 A31 A32 A33 A34 A35 A36 A37 A38 A39 A40 A41 A42 A43 A44 A45 A46 A47 A48 A49 A50 A51 A52 A53 A54 A55 A56 A57 A58 A59 A60 A61 A62 A63 A64 A65 A66 A67 A68 A69 A70 A71 A72 A73 A74 A75 A76 A77 A78 A79 A80 A81 A82 A83 A84 A85 A86 A87 A88 A89 A90 A91 A92 A93 A94 A95 A96 A97 A98 A99 A100 A101 A102 A103 A104 A105 A106 A107 A108 A109 A110 A111 A112 A113 A114 A115 A116 A117 A118 A119 A120 A121 A122 A123 A124 A125 A126 A127 A128 A129 A130 A131 A132 A133 A134 A135 A136 A137 A138 A139 A140 A141 A142 A143 A015'

In [0]:
registrations_df.at['1A57A4C0-7289-1014-A093-F050B7410E2F', 'regNum']

'A144-015'

In [0]:
# lots of other entries are reg number ranges, but this one seems like it COULD
# be the result of a keystroke error -- an errant hyphen, since its neighbors are
# near the same number ranges and 7 characters -- A148626 & A154653, for example

In [0]:
# so let's see if there are any others like this
len(registrations_df[(registrations_df['control_number'].str.contains('-')) & (registrations_df['control_number'].str.len() == 7)])

121

In [0]:
registrations_df[(registrations_df['control_number'].str.contains('-')) & (registrations_df['control_number'].str.len() == 7)].head()

,control_number,xml_year,author,title,edition,publisher,desc,control_date,regDate,copies,copyDate,affDate,regNum,lccn,note,newMatterClaimed,series,vol,volumes,claimant,prevPub,additionalEntry,prev-regNum,noticeDate,role
id,,,,,,,,,,,,,,,,,,,,,,,,,
1C0C0AC6-6E16-1014-BA0A-C4B2DFF7CF27,A-19989,1930,None,"My friend the black bass: with strategy, mecha...",NaN,None,"xiii p., 1 l., 288 p. front. (port.) 20ͨͫ.",1930-03-06,Mar. 6. 1930,2c.,NaN,Mar,A-19989,30-7626,with a frontispiece,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4AE21A8A-6CF9-1014-A6CF-A82867D68027,A0-7713,1933,None,Rizal and regaining our Eden lost,NaN,None,"xi, 111, [1] p. incl. port. 20½Cm",1933-12-22,Dec. 22. 1933,2c.,NaN,Dec. 27,A 0-7713,34-245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BBD61233-6C64-1014-8713-889FF4349C01,A0-9905,1934,None,The claimants,NaN,None,"4 p. l., 283 p. 19½ͨͫ",1934-03-14,"Mar. 14, 1934",2c.,NaN,Mar. 22,A 0-9905,34-5825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C81A4EFA-6C64-1014-8713-889FF4349C01,A7-1008,1934,None,"The golden precepts from the book of life, emb...",NaN,None,298 p. 24ͨͫ,1934-09-14,"Sept. 14, 1934",2c.,NaN,Sept. 24,A 7-1008,34-40127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9621DAC3-6D82-1014-B070-FDFD250A9EDA,A8-7772,1935,None,The Oakmont papers on immunology,NaN,None,"60 p. illus., diagrs. 23ͨͫ",1935-10-11,"Oct. 11, 1935",2c.,Oct. 16,Oct. 18,A 8-7772,35-30092,"Papers on edwenil therapy by various authors, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# these could certainly be errors. What about other len's?
len(registrations_df[(registrations_df['control_number'].str.contains('-')) & (registrations_df['control_number'].str.len() == 8)])

2624

In [0]:
registrations_df[(registrations_df['control_number'].str.contains('-')) & (registrations_df['control_number'].str.len() == 8)].head()

,control_number,xml_year,author,title,edition,publisher,desc,control_date,regDate,copies,copyDate,affDate,regNum,lccn,note,newMatterClaimed,series,vol,volumes,claimant,prevPub,additionalEntry,prev-regNum,noticeDate,role
id,,,,,,,,,,,,,,,,,,,,,,,,,
7FB36A48-6E18-1014-9400-B664922230AB,A-169025,1942,None,Used-to-be days.,NaN,NaN,NaN,1942-11-16,"Nov. 16, 1942",NaN,NaN,NaN,A-169025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00456B1E-7455-1014-8580-A6F8A5D8C9B4,A-182896,1944,None,Devious way,NaN,NaN,NaN,1944-09-15,"Sept. 15, 1944",NaN,NaN,NaN,A -182896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E2B93611-6E15-1014-9EF1-AF99805A40AD,AP0-6735,1952,None,L'abbé Constantin. Illus. de E. Dufour.,NaN,None,192 p.,1951-10-18,18Oct51,NaN,NaN,NaN,AP0-6735,NaN,NaN,on illus,(Bibliothèque verte),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8B0EDFE8-6F17-1014-9F0D-96810182DD4A,B5-24695,1953,None,THE ADVENTURES OF HAP HAZARD.,NaN,None,NaN,1953-07-13,13Jul,NaN,NaN,NaN,B5-24695,NaN,In the Co-operator,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
8BC38DA8-6F17-1014-9F0D-96810182DD4A,B5-24571,1953,None,McCarthy and anti-Semitism,NaN,None,NaN,1953-06-23,23Jun53,NaN,NaN,NaN,B5-24571,NaN,[Article],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
registrations_df[(registrations_df['control_number'] == 'A4-79813') | (registrations_df['control_number'] == 'A479813') | (registrations_df['regNum'] == 'A479813')
                 | (registrations_df['regNum'] == 'A479813')]

,control_number,xml_year,author,title,edition,publisher,desc,control_date,regDate,copies,copyDate,affDate,regNum,lccn,note,newMatterClaimed,series,vol,volumes,claimant,prevPub,additionalEntry,prev-regNum,noticeDate,role
id,,,,,,,,,,,,,,,,,,,,,,,,,
535E4CAA-6D03-1014-999C-964E71FFACD8,A479813,1961,None,Helping youth look at Jobs.,NaN,None,NaN,1960-08-31,31Aug60,NaN,NaN,NaN,A479813,NaN,(SRA special guidance report),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869C6A67-6D03-1014-999C-964E71FFACD8,A4-79813,1961,None,A collection of papers of the rational theory.,NaN,None,NaN,1960-12-28,28Dec60,NaN,NaN,NaN,A4-79813,NaN,Book 4,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN


In [0]:
registrations_df[(registrations_df['control_number'] == 'A144-015') | (registrations_df['control_number'] == 'A144015') |
                 (registrations_df['regNum'] == 'A144-015') | (registrations_df['regNum'] == 'A144015')]

,control_number,xml_year,author,title,edition,publisher,desc,control_date,regDate,copies,copyDate,affDate,regNum,lccn,note,newMatterClaimed,series,vol,volumes,claimant,prevPub,additionalEntry,prev-regNum,noticeDate,role
id,,,,,,,,,,,,,,,,,,,,,,,,,
07FB1489-70BF-1014-A0AD-E8B617DE6850,A144015,1940,None,"Poems, acrostics and songs.",NaN,None,NaN,1939-12-07,"Dec. 7, 1939",NaN,NaN,NaN,A 144015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1A57A4C0-7289-1014-A093-F050B7410E2F,A144 A16 A17 A18 A19 A20 A21 A22 A23 A24 A25 A...,1954,None,Some notes on autoclave curing for Portland ce...,NaN,None,5 l.,1954-05-20,20May54,NaN,NaN,NaN,A144-015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# at this point, it seems like that one was a processing error -- there are definitely ranges in the
# regNum field, but some registration numbers were robably improperly treated as ranges
# (has hypen, 7 or 8 characters)

In [0]:
# let's isolate these large-len entries and figure out if there are any more processing errors
temp = registrations_df[registrations_df['control_number'].str.len() > 200]
temp_2 = ((temp.reset_index(drop=False))[['id', 'control_number']]).to_dict('records')

In [0]:
# let's calculate the average size of each concatenated control number field
for entry in temp_2:
    numbers = entry['control_number'].split(' ')
    entry['avg'] = (sum(map(len, numbers)) / len(numbers))

In [0]:
temp_3 = [{'id': entry['id'], 'len': entry['avg']} for entry in temp_2 if entry['avg'] < 6]
pd.DataFrame(temp_3)

,id,len
0,1A57A4C0-7289-1014-A093-F050B7410E2F,3.353846


In [0]:
# that worked well. let's try it with a larger sample
temp = registrations_df[registrations_df['control_number'].str.len() > 50]
temp_2 = ((temp.reset_index(drop=False))[['id', 'control_number']]).to_dict('records')

for entry in temp_2:
    numbers = entry['control_number'].split(' ')
    entry['avg'] = (sum(map(len, numbers)) / len(numbers))
    
temp_3 = [{'id': entry['id'], 'len': entry['avg']} for entry in temp_2 if entry['avg'] < 6]
pd.DataFrame(temp_3)

,id,len
0,2A870394-6CB2-1014-A20E-B9D6251C946A,5.000000
1,86E5FBEF-6CFA-1014-9C3C-CEA95E7AA542,5.000000
2,875DD8C7-6CFA-1014-9C3C-CEA95E7AA542,5.000000
3,8BAA7768-6CFA-1014-9C3C-CEA95E7AA542,5.000000
4,92CBDE00-6CFA-1014-9C3C-CEA95E7AA542,5.000000
5,936EA631-6CFA-1014-9C3C-CEA95E7AA542,5.000000
6,597724E9-6E24-1014-A696-AD35B3FDE14F,4.000000
7,1A57A4C0-7289-1014-A093-F050B7410E2F,3.353846


In [0]:
# at the moment, the original one that stuck out looks like an anomaly. the others
# appear to be actual ranges of registration numbers, almost all of them in sequence, too

registrations_df.loc[['597724E9-6E24-1014-A696-AD35B3FDE14F', '936EA631-6CFA-1014-9C3C-CEA95E7AA542',
                      '8BAA7768-6CFA-1014-9C3C-CEA95E7AA542']]

,control_number,xml_year,author,title,edition,publisher,desc,control_date,regDate,copies,copyDate,affDate,regNum,lccn,note,newMatterClaimed,series,vol,volumes,claimant,prevPub,additionalEntry,prev-regNum,noticeDate,role
id,,,,,,,,,,,,,,,,,,,,,,,,,
597724E9-6E24-1014-A696-AD35B3FDE14F,A422 A423 A424 A425 A426 A427 A428 A429 A430 A...,1946,None,American law and procedure.,NaN,None,23 1/2ͨͫ.,1946-01-05,5Jan46,NaN,NaN,NaN,A422-A434,NaN,Andrews and v. 12–14 revised by the publisher'...,"Pub., revisions",NaN,Vols. 13–14,13 v,NaN,NaN,NaN,NaN,NaN,NaN
936EA631-6CFA-1014-9C3C-CEA95E7AA542,A6213 A6214 A6215 A6216 A6217 A6218 A6219 A622...,1929,None,Required poetry for reading and memorizing in ...,NaN,None,6 v. in 12. 19½ͨͫ,1928-04-27,"Apr. 27, 1928",2c.,"Mar. 27, 1929",Mar. 28,A 6213-6224,28-31126,[grade 1A-6B],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8BAA7768-6CFA-1014-9C3C-CEA95E7AA542,A4275 A4276 A4277 A4278 A4279 A4280 A4281 A428...,1929,None,Literature for reading and memorization for el...,NaN,None,12 v. 19ͨͫ,1929-01-17,"Jan. 17, 1929",1c.,Feb. 6,Feb. 4,"A 4275-4285, 5266",29-4659,"grades 1A-6A, 2B-6B",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# so now we need to do 2 things: (1) replace the [control_number] field at id '1A57A4C0-7289-1014-A093-F050B7410E2F' with ['regNum']
registrations_df.at['1A57A4C0-7289-1014-A093-F050B7410E2F', 'control_number'] = registrations_df.at['1A57A4C0-7289-1014-A093-F050B7410E2F', 'regNum']
registrations_df.at['1A57A4C0-7289-1014-A093-F050B7410E2F', 'control_number']

'A144-015'

In [0]:
# and (2) explode all the other [control_number] values that are separated by spaces
# current len of registrations_df:
len(registrations_df)

580261

In [0]:
registrations_df['control_number'] = registrations_df['control_number'].str.split()

In [0]:
for index, row in tqdm(registrations_df.iterrows()):
    if len(row['control_number']) == 1:
        registrations_df.at[index, 'control_number'] = row['control_number'][0]
    else:
        pass

580261it [01:05, 8907.99it/s]


In [0]:
registrations_df = registrations_df.explode('control_number')
len(registrations_df)

590344

In [0]:
# now let's check for any more registration numbers that need to be cleaned up and exploded again... looking for
# string lengths of 8 characters or more seems like a good check, considering that most reg IDs are 6-7 characters
len(registrations_df[registrations_df['control_number'].str.len() > 8])

58

In [0]:
registrations_df[registrations_df['control_number'].str.len() > 8]

,control_number,xml_year,author,title,edition,publisher,desc,control_date,regDate,copies,copyDate,affDate,regNum,lccn,note,newMatterClaimed,series,vol,volumes,claimant,prevPub,additionalEntry,prev-regNum,noticeDate,role
id,,,,,,,,,,,,,,,,,,,,,,,,,
1717D971-6E16-1014-BA0A-C4B2DFF7CF27,A.-Foreign,1930,None,… Novellen.,NaN,None,"208, [1] p., 1 l. 19ͨͫ.",1928-11-01,"Nov. 1, 1928",1c.,"Feb. 5, 1929",NaN,A.-Foreign 4644,30-5815,(On cover: Die polnische literatur der gegen-w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1737A33C-7289-1014-A093-F050B7410E2F,A157445125,1954,NaN,Modernize your kitchen.,NaN,None,32 p.,1954-10-01,1Oct54,NaN,NaN,NaN,A157445,NaN,in notice: Popular Mechanics Co.,NaN,Do-it-yourself booklet no. 15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173E85E5-7289-1014-A093-F050B7410E2F,A14568112729,1954,None,Daily devotions.,NaN,None,NaN,1954-06-07,7Jun,NaN,NaN,NaN,A145681,NaN,No.; 126. 6Jun-28Jul54. By Martin J. Zschoche ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17A1CDDA-72C4-1014-B53A-E905A29103D3,BB3866-3869,1957,None,The National Guard story.,NaN,None,NaN,1957-02-08,5Feb-8Feb57,NaN,NaN,NaN,BB3866-3869,NaN,"In Los Angeles evening herald & express, Feb. ...",NM: compilation.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18E246F3-7289-1014-A093-F050B7410E2F,AP0-15610,1954,None,Suite d'enseignements sur la thérapeutique hor...,NaN,None,222 p.,1954-10-15,15Oct54,NaN,NaN,NaN,AP0-15610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1A590743-6E16-1014-BA0A-C4B2DFF7CF27,A19206-A19298,1930,None,"Abbott New York digest … Consolidated ed., cov...",NaN,None,3 v. 26ͨͫ.,1930-01-27,"Jan. 27, 1930",2c.,Jan. 30,Jan. 31,A 19206-19298,NaN,CONTENTS.—v. 24. Marriage to Mayhem.—v. 25. Me...,NaN,NaN,v. 24-26 …,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1D54896F-6E16-1014-BA0A-C4B2DFF7CF27,A20521-A20535,1930,None,Abbott New York digest … Consolidated ed. 1929...,NaN,None,"40 v. 25"".",1930-02-01,"Feb. 1, 1930",2c.,NaN,Mar. 14,"A 20521-20535, 21090-21114",NaN,These pamphlets are to be fitted into the back...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1D54896F-6E16-1014-BA0A-C4B2DFF7CF27,A21090-21114,1930,None,Abbott New York digest … Consolidated ed. 1929...,NaN,None,"40 v. 25"".",1930-02-01,"Feb. 1, 1930",2c.,NaN,Mar. 14,"A 20521-20535, 21090-21114",NaN,These pamphlets are to be fitted into the back...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2262BB0C-715D-1014-A920-B56FE7FF3E29,A958825-958832,1927,[,The poetry book …,NaN,None,"col. fronts., illus. 19ͨͫ.",1926-12-31,"Dec. 31, 1926",2c.,"Jan. 22, 1927",NaN,NaN,27-1305,NaN,NaN,NaN,"v. 1, 3-9",9 v,NaN,NaN,None,NaN,NaN,NaN


In [0]:
# but there's also another problem: some registration IDs have been duplicated when they were processed by NYPL,
# or because they were included multiple times in the registration documents:
registrations_df.loc[['1D54896F-6E16-1014-BA0A-C4B2DFF7CF27']]

,control_number,xml_year,author,title,edition,publisher,desc,control_date,regDate,copies,copyDate,affDate,regNum,lccn,note,newMatterClaimed,series,vol,volumes,claimant,prevPub,additionalEntry,prev-regNum,noticeDate,role
id,,,,,,,,,,,,,,,,,,,,,,,,,
1D54896F-6E16-1014-BA0A-C4B2DFF7CF27,A20521-A20535,1930,None,Abbott New York digest … Consolidated ed. 1929...,NaN,None,"40 v. 25"".",1930-02-01,"Feb. 1, 1930",2c.,NaN,Mar. 14,"A 20521-20535, 21090-21114",NaN,These pamphlets are to be fitted into the back...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1D54896F-6E16-1014-BA0A-C4B2DFF7CF27,A21090-21114,1930,None,Abbott New York digest … Consolidated ed. 1929...,NaN,None,"40 v. 25"".",1930-02-01,"Feb. 1, 1930",2c.,NaN,Mar. 14,"A 20521-20535, 21090-21114",NaN,These pamphlets are to be fitted into the back...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# so we should fix the remaining registration numbers that need to be cleaned up, then explode the, 
# and finally de-dupe everything

# first, these should be fixed
registrations_df.at['1717D971-6E16-1014-BA0A-C4B2DFF7CF27', 'control_number'] = 'AF4644'
registrations_df.at['1737A33C-7289-1014-A093-F050B7410E2F', 'control_number'] = 'A157445'
registrations_df.at['173E85E5-7289-1014-A093-F050B7410E2F', 'control_number'] = 'A145681'
registrations_df.at['17A1CDDA-72C4-1014-B53A-E905A29103D3', 'control_number'] = 'BB3866 BB3867 BB3868 BB3869'
registrations_df.at['1A590743-6E16-1014-BA0A-C4B2DFF7CF27', 'control_number'] = 'A19206 A19298'
registrations_df.at['1D54896F-6E16-1014-BA0A-C4B2DFF7CF27', 'control_number'] = ('A20521 A20522 A20523 A20524 A20525 A20526 A20527 A20528 '
                                                                                 'A20529 A20530 A20531 A20532 A20533 A20534 A20535 A21090 '
                                                                                 'A21091 A21092 A21093 A21094 A21095 A21096 A21097 A21098 '
                                                                                 'A21099 A21100 A21101 A21102 A21103 A21104 A21105 A21106 '
                                                                                 'A21107 A21108 A21109 A21110 A21111 A21112 A21113 A21114')
registrations_df.at['2262BB0C-715D-1014-A920-B56FE7FF3E29', 'control_number'] = 'A958825 A958826 A958827 A958828 A958829 A958830 A958831 958832 A957862'
registrations_df.at['2671BE8B-72C4-1014-84F6-87703AD4166D', 'control_number'] = 'A346204 A353521'
registrations_df.at['271E1344-6F26-1014-8881-C801964016F5', 'control_number'] = 'A370932 A370933'
registrations_df.at['2F8C2AEA-715D-1014-A920-B56FE7FF3E29', 'control_number'] = 'A990443 A990444 A990445 A990446 A990447 A990448 A990449 A990450 A990451 A990452 A990453 A990454'
registrations_df.at['443D8C3D-798E-1014-9761-D3A611ADD299', 'control_number'] = 'A37589'
registrations_df.at['4CC78743-6D06-1014-A65A-D57CF5CB45BA', 'control_number'] = 'A1902-18'
registrations_df.at['5DB08207-6CFA-1014-B19C-B29A16CE9672', 'control_number'] = 'A505579 A505580'
registrations_df.at['5ED0BD3A-6D95-1014-B434-EB8C4702B141', 'control_number'] = 'A219700 A219701 A219702 A219703'
registrations_df.at['93BB2D13-6D0F-1014-9803-8BC52073E431', 'control_number'] = 'A414569 A414570 A414571 A414572'
registrations_df.at['940A806A-7673-1014-ACAA-B8C4DA013B26', 'control_number'] = 'A119580 A119581'
registrations_df.at['954F585A-76FE-1014-8A60-894D8A9620E2', 'control_number'] = 'A119514 A119515'
registrations_df.at['9558D741-6D0F-1014-9803-8BC52073E431', 'control_number'] = 'A28245'
registrations_df.at['96E03DA6-6D0F-1014-9803-8BC52073E431', 'control_number'] = 'A420869 A420870 A420871 A420872 A420873 A420874 A420875 A420876 A420877 A420878 A420879 A420880'
registrations_df.at['A466AB22-6E3A-1014-83DD-942C093A1DD9', 'control_number'] = 'A96935 A98447 A98810'
registrations_df.at['A5A227D9-6E3A-1014-83DD-942C093A1DD9', 'control_number'] = 'A100305 A100506 A100206'
registrations_df.at['C3407513-6C64-1014-8713-889FF4349C01', 'control_number'] = 'A69994 A70857'
registrations_df.at['C70EE45D-6C64-1014-8713-889FF4349C01', 'control_number'] = 'A74158 A73554'
registrations_df.at['C805EB6F-6C64-1014-8713-889FF4349C01', 'control_number'] = 'A78527 A78528'
registrations_df.at['C8619317-6C64-1014-8713-889FF4349C01', 'control_number'] = 'A77656 A77657'
registrations_df.at['D4DC8406-6D0A-1014-853A-B63123D40E62', 'control_number'] = 'A12598'
registrations_df.at['DCDF2753-6D0A-1014-853A-B63123D40E62', 'control_number'] = 'A18876 A13877'
registrations_df.at['E5DB6551-728D-1014-92CC-D4C993EBA1C2', 'control_number'] = 'B5-32309 B5-32350'
registrations_df.at['EEE0E824-7288-1014-A093-F050B7410E2F', 'control_number'] = 'A147511 A147513'
len(registrations_df)

590344

In [0]:
# next, explode them
registrations_df['control_number'] = registrations_df['control_number'].str.split()

for index, row in tqdm(registrations_df.iterrows()):
    if len(row['control_number']) == 1:
        registrations_df.at[index, 'control_number'] = row['control_number'][0]
    else:
        pass

590344it [01:13, 8032.37it/s]


In [0]:
registrations_df = registrations_df.explode('control_number')
len(registrations_df)

689408

In [0]:
# finally, dedupe
registrations_df.drop_duplicates(keep='first', inplace=True)
len(registrations_df)

580307

In [0]:
# even though we de-duped, we still have some duplicate indexed IDs:
registrations_df.index.duplicated().sum()

95

In [0]:
registrations_df[registrations_df.index.duplicated()][:3]

,control_number,xml_year,author,title,edition,publisher,desc,control_date,regDate,copies,copyDate,affDate,regNum,lccn,note,newMatterClaimed,series,vol,volumes,claimant,prevPub,additionalEntry,prev-regNum,noticeDate,role
id,,,,,,,,,,,,,,,,,,,,,,,,,
17A1CDDA-72C4-1014-B53A-E905A29103D3,BB3867,1957,None,The National Guard story.,NaN,None,NaN,1957-02-08,5Feb-8Feb57,NaN,NaN,NaN,BB3866-3869,NaN,"In Los Angeles evening herald & express, Feb. ...",NM: compilation.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17A1CDDA-72C4-1014-B53A-E905A29103D3,BB3868,1957,None,The National Guard story.,NaN,None,NaN,1957-02-08,5Feb-8Feb57,NaN,NaN,NaN,BB3866-3869,NaN,"In Los Angeles evening herald & express, Feb. ...",NM: compilation.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17A1CDDA-72C4-1014-B53A-E905A29103D3,BB3869,1957,None,The National Guard story.,NaN,None,NaN,1957-02-08,5Feb-8Feb57,NaN,NaN,NaN,BB3866-3869,NaN,"In Los Angeles evening herald & express, Feb. ...",NM: compilation.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# so we'll need new unique identifiers for each entity. let's use the uuid library
registrations_df.reset_index(drop=False, inplace=True)
registrations_df['uuid'] = registrations_df.apply(lambda x: uuid4().hex, axis=1)
registrations_df['uuid'].duplicated().sum()

0

In [0]:
registrations_df.set_index('uuid', inplace=True)

In [0]:
# now let's remove registrations without a registration number
registrations_df_cnums = registrations_df[(registrations_df['control_number'].notnull()) & (registrations_df['control_number'] != 'ERROR')]
len(registrations_df_cnums)

580162

In [0]:
# are any control dates empty?
len(registrations_df_cnums[registrations_df_cnums['control_date'].isnull()])

1330

In [0]:
# well that's a problem. Additionally, are any of the control dates not in YYYY-MM-DD format?
(registrations_df_cnums.sort_values(by=['control_date'])).head(8)

,id,control_number,xml_year,author,title,edition,publisher,desc,control_date,regDate,copies,copyDate,affDate,regNum,lccn,note,newMatterClaimed,series,vol,volumes,claimant,prevPub,additionalEntry,prev-regNum,noticeDate,role
uuid,,,,,,,,,,,,,,,,,,,,,,,,,,
89e64034fc5d4d6aadb4c5146b53cbf2,9271E7A0-6D17-1014-B416-88728C01EDA4,A544246,1962,NaN,ESSEX COUNTY (N. J.) REAL ESTATE DIRECTORY; re...,NaN,None,NaN,01,Jan,NaN,NaN,NaN,A544246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,",NaN,NaN,NaN
0fe0f49ed2044054ae9335c5789b1a16,9EF9D8F0-6D0F-1014-9803-8BC52073E431,A410549,1959,None,"Greener thumbs. July 6, 1959 through October 3...",NaN,None,NaN,0159,3,NaN,NaN,NaN,A410549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
617918245489497b8a00817d326c2dc1,C825EBA4-6D17-1014-B416-88728C01EDA4,A563336,1962,None,Telephone directories,NaN,None,NaN,03,Mar,NaN,NaN,NaN,A563336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015b9f4c4124b0d94199ca94f8ffe51,F52190C1-6D13-1014-B63C-9736EB68D6D5,A578167,1962,None,The colonial image; origins of American cultur...,NaN,None,NaN,04-02,2Aprb2,NaN,NaN,NaN,A578167,NaN,NaN,"NM: pref., introd. & connecting notes",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3b6de506eca6496a9530c7cd46341bef,9E9760C3-6D0F-1014-9803-8BC52073E431,A396269,1959,None,Statutes.,NaN,None,NaN,04-28,28Apr,NaN,NaN,NaN,A396269,NaN,NaN,NaN,NaN,"v.40, no. 3",NaN,NaN,NaN,NaN,NaN,NaN,NaN
c841e5c8b8024a16b5f81be1344b14b0,C477BE74-6D13-1014-B63C-9736EB68D6D5,A587712,1962,None,Electric eye movie manual.,NaN,None,NaN,05-06,21May6l,NaN,NaN,NaN,A587712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02e99574f9c340538ff5d5ff205da972,9E988846-6D0F-1014-9803-8BC52073E431,A396272,1959,None,Cases and statutes.,NaN,None,NaN,05-07,7May,NaN,NaN,NaN,A396272,NaN,NaN,NaN,NaN,"v.53, no. 2, May59",NaN,NaN,NaN,"3,",NaN,NaN,NaN
43ccfa3079704d768f20c0a2938e75ce,9EFE6BCF-6D0F-1014-9803-8BC52073E431,A402926,1959,None,International Bible lesson column.,NaN,None,NaN,05-11,11May,NaN,NaN,NaN,A402926,NaN,For 24May-28Jun59,NaN,NaN,NaN,NaN,NaN,NaN,20Dec59-24Jan60. ©,NaN,NaN,NaN


In [0]:
# well that's kind of a problem too. let's set these to null, otherwise it'll mess up
# our matching attempts later on. We have the year of registration from the XML data,
# so we'll construct an identifier with the control number and the year of registration
# in a little bit

In [0]:
# loop through rows matching dates to a regex; if no match, set to NaN
for index, row in tqdm(registrations_df_cnums.iterrows()):
    if (pd.isnull(row['control_date'])):
        pass
    else:
        prog = re.compile(r"^\d{4}\-\d{2}\-\d{2}$")
        result = prog.match(str(row['control_date']))
        if result:
            pass
        else:
            registrations_df_cnums.at[index, 'control_date'] = np.NaN

580162it [00:57, 10168.63it/s]


In [0]:
# now, how many control dates are empty?
len(registrations_df_cnums[registrations_df_cnums['control_date'].isnull()])

1570

In [0]:
# we'll separate the ones without a reg date into their own DF
registrations_df_cnums_no_dates = registrations_df_cnums[registrations_df_cnums['control_date'].isnull()]
len(registrations_df_cnums_no_dates)

1570

In [0]:
# and make a new DF for not-null control date records
registrations_df_cnums_dates = registrations_df_cnums[registrations_df_cnums['control_date'].notnull()]
len(registrations_df_cnums_dates)

578592

In [0]:
# now let's create unique identifiers for the date/nodate DFs
# cdmid = cross-dimensional matching identifier (with control date)
registrations_df_cnums_dates['cdmid'] = registrations_df_cnums_dates.apply(lambda x: x['control_number'] + '|' + x['control_date'], axis=1)

/home/scottythered/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
# cdmid_nd = cross-dimensional matching identifier (no control date)
registrations_df_cnums_no_dates['cdmid_nd'] = registrations_df_cnums_no_dates.apply(lambda x: x['control_number'] + '|' + x['xml_year'], axis=1)

/home/scottythered/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# To sum up:

- **registrations_df** --> all registration records
- **registrations_df_cnums** --> all registrations that have control numbers
- **registrations_df_cnums_no_dates** --> all registrations with control numbers but no dates, just years (cdmid_nd)
- **registrations_df_cnums_dates** --> all registrations with control numbers and dates (cdmid)

# 2. Copyright Renewals

## Again, some suppositions:

1. "Often more than one registration is involved, such as when a book is first published outside the United States and has an 'interim' registration (class AI) before it's final registration. \[...\] Renewal *R129296* refers to both the interim registration, *AI-9217*, and final registration, *A972756*. This illustrates the importance of the id numbers, since the two original registrations have two different titles and wouldn't be clear otherwise that they are the same book. This allows us to easily match the two registrations found in the Registrations data."

2. Again, copyright registrations that happened after 1963 were renewed automatically, so there's no point in matching for them.

3. Also, copyright registrations that happened *before* 1924 are currently in the public domain, so there's no point in matching those either.

In [0]:
# load up directory of copyright renewals coded as tab-separated docs
renewals_directory = os.path.join(base_path, 'cce-renewals-master', 'data')
renewals_files = [x for x in os.listdir(renewals_directory) if x[-4:] == '.tsv']
renewals_df = pd.DataFrame()
for file in tqdm(renewals_files):
    temp_df = pd.read_csv(os.path.join(renewals_directory, file), sep='\t', header=0)
    renewals_df = renewals_df.append(temp_df, sort=False, ignore_index=True)

100%|██████████| 47/47 [00:12<00:00,  1.70it/s]


In [0]:
# some later TSVs have an incorrect version of the title header...
temp = (renewals_df[renewals_df['titl'].notnull()]).index.tolist()
temp[:5]

[175931, 175932, 175933, 175934, 175935]

In [0]:
# ...and the same TSVs have an incorrect version of author
temp_2 = (renewals_df[renewals_df['auth'].notnull()]).index.tolist()
temp_2[:5]

[175931, 175932, 175933, 175934, 175935]

In [0]:
# move these pieces of data to the correct columns...
for index, row in tqdm(renewals_df.iterrows()):
    if index >= 12960:
        renewals_df.at[index, 'title'] = row['titl']
        renewals_df.at[index, 'author'] = row['auth']
    else:
        pass

445386it [00:51, 8716.62it/s]


In [0]:
# ...then drop the incorrect heading columns
renewals_df.drop(['titl', 'auth'], axis=1, inplace=True)

In [0]:
# Get indexes for rows where copyright registrations happened after 1963 or before 1924
temp = renewals_df[renewals_df['odat'].notnull()]
temp['year'] = temp.apply(lambda x: int((x['odat'].split('-'))[0]), axis=1)
indexes = temp[(temp['year'] < 1924) | (temp['year'] > 1963)].index

/home/scottythered/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
len(indexes)

8154

In [0]:
len(renewals_df)

445386

In [0]:
# Delete these roww from the dataFrame
renewals_df.drop(indexes, inplace=True)
len(renewals_df)

437232

In [0]:
# now then, do we have duplicate entity ids?
renewals_df['entry_id'].duplicated().sum()

13738

In [0]:
# sigh
renewals_df[renewals_df['entry_id'].duplicated()].sort_values('entry_id')[3:6]

,entry_id,volume,part,number,page,author,title,oreg,odat,id,rdat,claimants,new_matter,see_also_ren,see_also_reg,notes,full_text,dreg,note
18193,000dfedf-e0da-5c46-b700-b1bd0bd78f13,8.0,1,2.0,1531.0,NaN,NaN,B754474,1927-10-06,R138115,1954-10-22,Dorothy Benjamin Caruso & Torrance Goddard|A,NaN,NaN,NaN,NaN,"CARUSO, DOROTHY BENJAMIN. Wings of song, by Do...",NaN,NaN
18195,000dfedf-e0da-5c46-b700-b1bd0bd78f13,8.0,1,2.0,1531.0,NaN,NaN,B755955,1927-10-20,R138117,1954-10-22,Dorothy Benjamin Caruso & Torrance Goddard|A,NaN,NaN,NaN,NaN,"CARUSO, DOROTHY BENJAMIN. Wings of song, by Do...",NaN,NaN
18194,000dfedf-e0da-5c46-b700-b1bd0bd78f13,8.0,1,2.0,1531.0,NaN,NaN,B754640,1927-10-13,R138116,1954-10-22,Dorothy Benjamin Caruso & Torrance Goddard|A,NaN,NaN,NaN,NaN,"CARUSO, DOROTHY BENJAMIN. Wings of song, by Do...",NaN,NaN


In [0]:
# okay, we'll need to take care of that by adding UUIDs
renewals_df['uuid'] = renewals_df.apply(lambda x: uuid4().hex, axis=1)
renewals_df['uuid'].duplicated().sum()

0

In [0]:
# we'll also need to do a check for any multi-renewal numbers in the 'oreg' field
renewals_df['oreg'].str.contains(' ').sum()

1345

In [0]:
# sigh
renewals_df[(renewals_df['oreg'].str.contains(' ')) & (renewals_df['oreg'].notnull())]

,entry_id,volume,part,number,page,author,title,oreg,odat,id,rdat,claimants,new_matter,see_also_ren,see_also_reg,notes,full_text,dreg,note,uuid
1536,200047e3-0519-5307-9ac6-98c156fd3588,4.0,14A,1.0,35.0,NaN,MAYFLOWER. (Butter),Label 25885,1925-03-13,R60206,1950-03-29,"Pickerington Creamery, Inc.|P",NaN,NaN,NaN,NaN,"MAYFLOWER. (Butter) © 13Mar25, Label 25885. R6...",NaN,NaN,d46d065e70634a44a13f7ed7c63e98d9
205880,aea39bc9-b4c6-153e-d432-b7b53e193c66,NaN,NaN,NaN,NaN,NaN,The Hills of Korea. Words & music: Joseph C. ...,B00000000005 20688,1952-03-26,RE46245,NaN,Joseph C. Murphy|A,NaN,NaN,NaN,NaN,NaN,1980-01-08,NaN,d90eb15f3efa4f6fbaa3765662498852
294954,45807b0f-2bc6-51ec-d96a-ccc1cad0588d,NaN,NaN,NaN,NaN,NaN,Rio Batucada. English lyrics by John Wedgwood...,UCC work,1956-12-11,RE407262,NaN,Nezika Connelly|W,NaN,NaN,NaN,NaN,NaN,1984-12-20,NaN,1f0712498eb643d4842322af335b14fd
296052,3322d17b-efb8-9aa8-21d3-f13cfc33e036,NaN,NaN,NaN,NaN,NaN,The Craft of fiction. Foreword and editorial:...,UCC work,1957-01-01,RE260995,NaN,John Pym|NK,NaN,NaN,NaN,NaN,NaN,1985-10-04,NaN,5d699df5269045de88700901bbec3801
297963,75919c10-ac72-9984-94fc-303386e76e62,NaN,NaN,NaN,NaN,NaN,Selected poems of William Blake. Introd. and ...,UCC work,1957-02-01,RE268736,NaN,F. W. Bateson|A,NaN,NaN,NaN,NaN,NaN,1985-12-23,NaN,a991db81d0424135bfb632a21ceda67d
297964,946a16c4-cb55-3bb1-fcd0-08b872a91fae,NaN,NaN,NaN,NaN,NaN,Paris und Helena; Musikdrama in fuenf Akten. ...,UCC work,1957-02-01,RE272492,NaN,Friedrich Wilhelm Lothar|A,NaN,NaN,NaN,NaN,NaN,1985-12-18,NaN,162fb460359240b6a2cb6d0ac7b7c326
299471,13f6400d-d338-0fd4-6e95-1d2483159596,NaN,NaN,NaN,NaN,Idries Shah.,The Secret lore of magic.,UCC work,1957-03-01,RE261755,NaN,Idries Shah|A,NaN,NaN,NaN,NaN,NaN,1985-10-16,NaN,3d96326a63aa4449b5e0495d6ec62f4a
299472,c6172eef-ee81-2810-aeeb-de59bb6ccc6c,NaN,NaN,NaN,NaN,D. S. L. Cardwell.,The Organisation of science in England.,UCC work,1957-03-01,RE268734,NaN,D. S. L. Cardwell|A,NaN,NaN,NaN,NaN,NaN,1985-12-23,NaN,a711d1165b64400b8bd4e59a9f00f0ac
299473,09ff17b7-84b9-65d2-c01f-8808a61d15cd,NaN,NaN,NaN,NaN,Guenter Bialas.,Abschiedslied eines Maedchens. Fuer vierstimm...,UCC work,1957-03-01,RE272489,NaN,Guenter Bialas|A,NaN,NaN,NaN,NaN,NaN,1985-12-18,NaN,be97522221eb48b6933261650b3b97fc
299474,20f5c51f-88cb-20d5-1e24-b6ebf61d0cb3,NaN,NaN,NaN,NaN,Guenter Bialas.,Brautlied. Fuer vierstimmigen Chor.,UCC work,1957-03-01,RE272488,NaN,Guenter Bialas|A,NaN,NaN,NaN,NaN,NaN,1985-12-18,NaN,142ba6ac34224461a52ca1f4e114e5c4


In [0]:
# lets fix that second one
renewals_df.at[205880, 'oreg'] = 'B520688'

In [0]:
# i don't know what 'UCC work' means. they're not useful, so let's set those to blank
blanked = (renewals_df[(renewals_df['oreg'].str.contains('UCC work')) & (renewals_df['oreg'].notnull())]).index.to_list()
len(blanked)

1321

In [0]:
for index in tqdm(blanked):
    renewals_df.at[index, 'oreg'] = np.NaN

100%|██████████| 1321/1321 [00:00<00:00, 46854.42it/s]


In [0]:
# check it again
renewals_df[(renewals_df['oreg'].str.contains(' ')) & (renewals_df['oreg'].notnull())]

,entry_id,volume,part,number,page,author,title,oreg,odat,id,rdat,claimants,new_matter,see_also_ren,see_also_reg,notes,full_text,dreg,note,uuid
1536,200047e3-0519-5307-9ac6-98c156fd3588,4.0,14A,1.0,35.0,NaN,MAYFLOWER. (Butter),Label 25885,1925-03-13,R60206,1950-03-29,"Pickerington Creamery, Inc.|P",NaN,NaN,NaN,NaN,"MAYFLOWER. (Butter) © 13Mar25, Label 25885. R6...",NaN,NaN,d46d065e70634a44a13f7ed7c63e98d9
335222,9f560931-8bd9-d974-2423-ed395061d365,NaN,NaN,NaN,NaN,B. R. Nanda.,Mahatma Gandhi; a biography.,U.C.C. work,1958-12-01,RE344004,NaN,B. R. Nanda|A,NaN,NaN,NaN,NaN,NaN,1986-11-03,NaN,c173154dac1d4e9cbfeb4c5c96ed36a7
413267,c5b73508-10bd-a863-33c8-36433abbcc6a,NaN,NaN,NaN,NaN,NaN,The Statesman's year-book: statistical and his...,U.C.C. work,1962-08-09,RE512474,NaN,"Macmillan Academic & Professional, Ltd.|PCW",NaN,NaN,NaN,NaN,NaN,1990-12-31,NaN,8f03e3ca44124bb4823bc5550ef7d2ba
415596,c92f168d-8e7e-2436-5b49-30b085b788ad,NaN,NaN,NaN,NaN,"Henry Cecil, pseud. of Henry Cecil Leon.",Unlawful occasions.,USC work,1962-09-17,RE487528,NaN,Barbara Leon|W,NaN,NaN,NaN,NaN,NaN,1990-05-07,NaN,e0e8ce83e197436f88ac1ecb4de2455a
418257,75d22f77-8676-1a1a-f668-0b897a48faec,NaN,NaN,NaN,NaN,NaN,Inflation.,U.C.C. work,1962-10-25,RE512475,NaN,International Economic Association|PCW,NaN,NaN,NaN,NaN,NaN,1990-12-31,NaN,ce5fa8541d8d4882b3e351ed8be98b9c
421788,36a09665-46c5-d56a-b9ee-c5b058aee15c,NaN,NaN,NaN,NaN,"Leo Bruce, pseud. of Rupert Croft-Cooke.",Nothing like blood.,UCC renewal,1962-12-30,RE512034,NaN,Richard Hitchcock|NK,NaN,NaN,NaN,NaN,NaN,1990-12-17,NaN,7d9c6bd19acc418ab5814a49d4db2a51
423528,d6a1515a-56c2-ea60-40fe-74161dc500f3,NaN,NaN,NaN,NaN,NaN,On the beat. m Philip Green.,U.C.C. work,1963-01-24,RE557645,NaN,Dorothy Mary Green|W,NaN,NaN,NaN,NaN,NaN,1991-12-24,NaN,c9eeab8f55ac4dfbb91b4caa9348e675
424331,6d0e3366-a008-783a-7533-6f6c6a685049,NaN,NaN,NaN,NaN,NaN,Panatella. m Ron Goodwin.,U.C.C. work,1963-02-07,RE557644,NaN,Ron Goodwin|A,NaN,NaN,NaN,NaN,NaN,1991-12-24,NaN,f566669a104749bba6f1631f4ea33c64
429682,91d92297-9883-b931-c1bc-7d4a4caa2cb8,NaN,NaN,NaN,NaN,NaN,"Bless your heart, my darling. w & m Alex Alst...",U.C.C. work,1963-04-29,RE557653,NaN,Alex Alstone|A||Derek James Kennedy|C,NaN,NaN,NaN,NaN,NaN,1991-12-23,NaN,52b5ff0f628c4f32936e62f8dd157edb
430070,185856a0-8a42-6ce2-4198-9f13a610594d,NaN,NaN,NaN,NaN,Dick Sadleir.,Rhythm and blues guitar solos.,U.C.C. work,1963-05-03,RE557652,NaN,L. H. Sadleir|W,NaN,NaN,NaN,NaN,NaN,1991-12-23,NaN,4c1f1d8dc1ca4386852a18302c2e7543


In [0]:
# finish it up with a second round
blanked_2 = (renewals_df[(renewals_df['oreg'].str.contains(' ')) & (renewals_df['oreg'].notnull())][1:]).index.to_list()
for index in tqdm(blanked_2):
    renewals_df.at[index, 'oreg'] = np.NaN

100%|██████████| 22/22 [00:00<00:00, 50423.33it/s]


In [0]:
# next, any duplicates?
renewals_df.duplicated().sum()

0

In [0]:
# now let's find all records missing oregs and remove them
len(renewals_df[renewals_df['oreg'].isnull()])

3037

In [0]:
renewals_df_cnums = renewals_df[renewals_df['oreg'].notnull()]
len(renewals_df_cnums)

434195

In [0]:
# any missing reg dates?
len(renewals_df_cnums[renewals_df_cnums['odat'].isnull()])

21

In [0]:
renewals_df_cnums[renewals_df_cnums['odat'].isnull()]

,entry_id,volume,part,number,page,author,title,oreg,odat,id,rdat,claimants,new_matter,see_also_ren,see_also_reg,notes,full_text,dreg,note,uuid
71601,d329cd30-9842-5067-88fb-d5ef20b7e984,18.0,1,2.0,2224.0,NaN,NaN,A5-82665,NaN,R346845,1964-10-19,Walter E. Disney|A,NaN,NaN,NaN,NaN,"DISNEY, WALTER E. Donald Duck, by Walt Disney....",NaN,NaN,061508f7a6044d8d8c0402756b369411
113289,d10e51ac-acfa-532d-a823-78d9323e83df,24.0,1,1.0,1433.0,NaN,NaN,B552877,NaN,R476844,1970-01-16,Walt Coburn|A,NaN,NaN,NaN,NaN,"COBURN, WALT. Dower rights to Gunsmoke Ranch. ...",NaN,NaN,f4b612128efd4ccbbe083370e7673ca8
136373,8b69828f-e476-598c-8c6a-2c93a473fd04,27.0,1,1.0,1809.0,NaN,NaN,AA480113,NaN,R544382,1973-01-19,John W. Schaum|A,NaN,NaN,NaN,NaN,"SCHAUM, JOHN W., comp. John W. Schaum note spe...",NaN,NaN,67c5890254ab43daa446f8e59b627d5e
182658,076b1995-25ba-e997-1764-07a74ffe7cfd,NaN,NaN,NaN,NaN,NaN,"The Nation. Vol. 171, no. 23, Dec. 2, 1950.",B279472,NaN,RE10320,NaN,"The Nation Associates, Inc.|PCW",NaN,NaN,NaN,NaN,NaN,1978-12-18,NaN,b2fe21ce47b248d3b57ec4621e9c5298
189765,2e782446-9844-1d15-28aa-c9f6d4721033,NaN,NaN,NaN,NaN,Lee Hendry.,'Twas the night before Christmas; a one act play.,DP682,NaN,RE17937,NaN,"Samuel French, Inc.|PWH",NaN,NaN,NaN,NaN,NaN,1979-01-16,NaN,25379ca5e0ad45349688df3baa61d399
190008,ec08bcbe-ea46-4ef0-66c6-a3edb86a5510,NaN,NaN,NaN,NaN,Truman Capote.,Botteghe oscure|The Grass harp.,A59474,NaN,RE54289,NaN,Truman Capote|A,NaN,NaN,NaN,NaN,NaN,1979-12-17,First chapter of under ti.: The Grass harp app...,ef8c5a27b6a446faaba00e45312bf4b5
252266,70dcb87a-8684-6f46-159d-f2b52d5f775f,NaN,NaN,NaN,NaN,NaN,"General digest, second series. Vol. 24, no. 4...",B499047,NaN,RE137594,NaN,West Publishing Company|PWH,NaN,NaN,NaN,NaN,NaN,1982-10-01,NaN,8af32a36ccbe4e50801db27fe0fe6b2e
279715,287b2ee1-a957-845f-bd46-8545b6158fb2,NaN,NaN,NaN,NaN,NaN,"Radiation research. Vol. 4, no. 1, Jan. 1956.",B580161,NaN,RE222859,NaN,"Academic Press, Inc.|PCW",NaN,NaN,NaN,NaN,NaN,1984-12-11,NaN,dc97c0b8dc2a4edc9ed3a1b4aaaa6370
309262,f7281d65-80eb-7618-c206-9ca7d5309f1b,NaN,NaN,NaN,NaN,Ruth Seder Cooke.,"Vogue patterns; suppl. Oct. 15 collection, 1957.",A301144,NaN,RE271598,NaN,"Conde Nast Publications, Inc.|PWH",NaN,NaN,NaN,NaN,NaN,1985-12-23,NaN,4a27792c244343f0993af11d4b64bf9b
311072,593b7d30-772e-5367-5f95-161e429d02c0,NaN,NaN,NaN,NaN,NaN,The New Navy mobile power for peace.,A312386,NaN,RE244854,NaN,United States Naval Institute|PWH,NaN,NaN,NaN,NaN,NaN,1985-04-18,NaN,ca2801def318417e8db28f36f98fae8f


In [0]:
# remove them from the DF
renewals_df_cleaned = renewals_df_cnums[renewals_df_cnums['odat'].notnull()]

In [0]:
len(renewals_df_cleaned)

434174

In [0]:
# quick check to make sure all odates are correctly formed
(renewals_df_cleaned.sort_values(by=['odat'])).head(5)

,entry_id,volume,part,number,page,author,title,oreg,odat,id,rdat,claimants,new_matter,see_also_ren,see_also_reg,notes,full_text,dreg,note,uuid
8843,09cff772-4d5c-5251-a3db-3ffca339d09c,5.0,1A,2.0,658.0,NaN,"SWARTWOOD, CHARLES C. My Mayday queen.",A776398,1924-01-01,R86928,1951-12-03,Charles C. Swartwood|A,NaN,NaN,NaN,NaN,"SWARTWOOD, CHARLES C. My Mayday queen. © 1Jan2...",NaN,NaN,16a167bc41dd47758838deb961f2cffa
8119,1c8e5597-9ff8-50ee-807d-48731ce1feac,5.0,1A,2.0,641.0,NaN,"FENTON, CARROLL LANE. The age of mammals. (Lit...",A777274,1924-01-01,R87794,1951-12-29,Haldeman-Julius Co.|PWH,NaN,NaN,NaN,NaN,"FENTON, CARROLL LANE. The age of mammals. (Lit...",NaN,NaN,37083e053c80406bbc3278e0d7221997
8914,217d2d3e-c8f0-50c1-aa42-3ca6abfb0499,5.0,1A,2.0,659.0,Otto A. Horsack.,UNION SPECIAL MACHINE COMPANY. Samples of doub...,A773769,1924-01-01,R83813,1951-09-28,Union Special Machine Co.|PWH,NaN,NaN,NaN,NaN,UNION SPECIAL MACHINE COMPANY. Samples of doub...,NaN,NaN,2663049e844a4d2fa213468b35ac871b
7046,411694c9-6fc5-5a47-9f8f-7c8ff42d956e,5.0,1A,1.0,251.0,NaN,"GEISTER, EDNA. Breaking the ice in January. (T...",A776238,1924-01-01,R75329,1951-02-26,Edna Geister|A,NaN,NaN,NaN,NaN,"GEISTER, EDNA. Breaking the ice in January. (T...",NaN,NaN,66c13ef9dd0f4615836b457163bb2600
8913,3919bb25-1b99-536a-9337-f8019165c302,5.0,1A,2.0,659.0,Otto A. Horsack.,UNION SPECIAL MACHINE COMPANY. Samples of doub...,A773767,1924-01-01,R83811,1951-09-28,Union Special Machine Co.|PWH,NaN,NaN,NaN,NaN,UNION SPECIAL MACHINE COMPANY. Samples of doub...,NaN,NaN,d07b840c21954509a750bd7cbe2a8e6b


In [0]:
renewals_df_cleaned.sort_values(by=['odat'], ascending=False).head(5)

,entry_id,volume,part,number,page,author,title,oreg,odat,id,rdat,claimants,new_matter,see_also_ren,see_also_reg,notes,full_text,dreg,note,uuid
445265,7e2bbfb3-fa32-65e3-f964-2775c9866bd5,NaN,NaN,NaN,NaN,Harold W. Rusch and Frank Scimonelli.,Advanced band method|B-flat cornet & trumpet.,A700221,1963-12-31,RE538905,NaN,Hal Leonard Publishing Corporation|PWH,NaN,NaN,NaN,NaN,NaN,1991-07-22,NaN,61f13f6b01b84eb798098f7c7b82bd17
445239,325dc00d-a05f-765d-beca-12c797f37e15,NaN,NaN,NaN,NaN,John Polgreen & Cathleen Polgreen.,"Our friend, the sun.",A682549,1963-12-31,RE547382,NaN,"Henry Holt and Company, Inc.|PWH",NaN,NaN,NaN,NaN,NaN,1991-10-04,NaN,17cd99458b584afabf26ec3047a0e6dc
445259,6b8479b2-adb1-411c-c294-60f9c469ba1a,NaN,NaN,NaN,NaN,NaN,Perspectives in medieval history. Editors: Ka...,A698997,1963-12-31,RE548363,NaN,William Marsh Rice University|PCW,NaN,NaN,NaN,NaN,NaN,1991-10-23,NaN,03c02270a5ea484fab04a7c8c4bfa86c
445258,0781bc78-04cf-3c03-14f7-e1bebcfbf3ca,NaN,NaN,NaN,NaN,Richard Estes.,Fossil vertebrates from the late Cretaceous La...,A696286,1963-12-31,RE565750,NaN,Richard Estes|A,NaN,NaN,NaN,NaN,NaN,1991-12-31,NaN,e44a7f6414064b34a1ce68a4ec52ac6a
445257,6f7d3e78-61dd-cb19-5915-7ff62f3a9dd7,NaN,NaN,NaN,NaN,NaN,Adventures in public service. Editors: Delia ...,A691919,1963-12-31,RE548352,NaN,Delia Kuhn|A,NaN,NaN,NaN,NaN,NaN,1991-10-15,NaN,3083a8c1793a46b6b2101364029e073a


In [0]:
# now let's create unique identifiers for the DFs
renewals_df_cleaned['cdmid'] = renewals_df.apply(lambda x: str(x['oreg']) + '|' + str(x['odat']), axis=1)

/home/scottythered/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
renewals_df_cleaned['cdmid_nd'] = renewals_df.apply(lambda x: str(x['oreg']) + '|' + (str(x['odat']).split('-')[0]), axis=1)

/home/scottythered/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
len(renewals_df_cleaned)

434174

In [0]:
renewals_df_cleaned.head()

,entry_id,volume,part,number,page,author,title,oreg,odat,id,rdat,claimants,new_matter,see_also_ren,see_also_reg,notes,full_text,dreg,note,uuid,cdmid,cdmid_nd
122,6efc89b0-7542-5e93-8207-69e1900bb0ef,4.0,14A,1.0,3.0,E. F. Benson,AT THE FARMHOUSE; later included in VISIBLE AN...,A778640,1924-03-28,R59144,1950-02-23,Kenneth Stewart Patrick McDowall|NK,NaN,NaN,NaN,"pub. abroad 20Feb23, AI-4984","AT THE FARMHOUSE, by E. F. Benson; later inclu...",NaN,NaN,03609b71da864fcc9a6319e58782cb1c,A778640|1924-03-28,A778640|1924
753,c3ef9a6f-8355-5961-9aeb-79715a323766,4.0,14A,1.0,18.0,Richard Connell.,"FISTS (In the Saturday evening post, Mar. 10, ...",B571853,1925-03-08,R61904,1950-05-04,Louise Fox Connell|W,NaN,NaN,NaN,NaN,"FISTS, by Richard Connell. (In the Saturday ev...",NaN,NaN,2af515ad21af405a94b45534ca94163a,B571853|1925-03-08,B571853|1925
1057,9f623d7a-1c30-5a31-be18-76e452743ef2,4.0,14A,1.0,25.0,E. F. Benson,INSCRUTABLE DECREES; later included in VISIBLE...,A778640,1924-03-28,R59866,1950-03-16,Kenneth Stewart Patrick McDowall|NK,NaN,NaN,NaN,"pub. abroad 14Mar23, AI-5023","INSCRUTABLE DECREES, by E. F. Benson; later in...",NaN,NaN,ccc034a864fb40a68f099fafbbbddb96,A778640|1924-03-28,A778640|1924
1105,bdf4a5d8-3d80-54de-8c75-2601d08e0284,4.0,14A,1.0,26.0,NaN,"INSURANCE MAPS. Edinburgh, Christian County, I...",F42232,1925-04-10,R62490,1950-05-17,Sanborn Map Co.|PWH,NaN,NaN,NaN,NaN,INSURANCE MAPS. © Sanborn Map Co. (PWH) Edinbu...,NaN,NaN,2e0b51f7c4174e4db9c847386e3a0433,F42232|1925-04-10,F42232|1925
1200,64ed4644-14c2-54ad-85da-8d0b218921c6,4.0,14A,1.0,27.0,NaN,"IT'S NEVER TOO LATE, a comedy in three acts by...",D64453,1925-05-12,R63230,1950-06-08,Elizabeth Glass Wardell|C||Jules Eckert Goodman|A,NaN,NaN,NaN,NaN,"IT'S NEVER TOO LATE, a comedy in three acts by...",NaN,NaN,7f8d7aae0c504c41b27dc969a0cd4f2b,D64453|1925-05-12,D64453|1925


In [0]:
with shelve.open('shelf.p', 'c') as shelf:
    shelf['registrations_df'] = registrations_df
    shelf['registrations_df_cnums'] = registrations_df_cnums
    shelf['registrations_df_cnums_no_dates'] = registrations_df_cnums_no_dates
    shelf['registrations_df_cnums_dates'] = registrations_df_cnums_dates
    shelf['renewals_df'] = renewals_df
    shelf['renewals_df_cnums'] = renewals_df_cnums
    shelf['renewals_df_cleaned'] = renewals_df_cleaned

In [0]:
with shelve.open('shelf.p', 'r') as shelf:
    registrations_df = shelf['registrations_df']
    registrations_df_cnums = shelf['registrations_df_cnums']
    registrations_df_cnums_no_dates = shelf['registrations_df_cnums_no_dates']
    registrations_df_cnums_dates = shelf['registrations_df_cnums_dates']
    renewals_df = shelf['renewals_df']
    renewals_df_cnums = shelf['renewals_df_cnums']
    renewals_df_cleaned = shelf['renewals_df_cleaned']

# Another Sum-Up

- **renewals_df** --> all renewal records
- **renewals_df_cnums** --> all renewal records with control numbers
- **renewals_df_cleaned** --> all renewal records with registration dates and registration control numbers (cdmid and cdmid_nd)

# Matching
## Pass One

In [0]:
# isolate renewals DF for cdmids
renewals_cdmid = renewals_df_cleaned[['uuid', 'cdmid']]
renewals_cdmid.rename(columns={"uuid": "renewal_uuid"}, inplace=True)

/home/scottythered/.local/lib/python3.6/site-packages/pandas/core/frame.py:4218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [0]:
renewals_cdmid.head()

,renewal_uuid,cdmid
122,03609b71da864fcc9a6319e58782cb1c,A778640|1924-03-28
753,2af515ad21af405a94b45534ca94163a,B571853|1925-03-08
1057,ccc034a864fb40a68f099fafbbbddb96,A778640|1924-03-28
1105,2e0b51f7c4174e4db9c847386e3a0433,F42232|1925-04-10
1200,7f8d7aae0c504c41b27dc969a0cd4f2b,D64453|1925-05-12


In [0]:
# merge to match cdmids
registrations_df_cnums_dates.reset_index(drop=False, inplace=True)
cdmid_matches = registrations_df_cnums_dates.merge(renewals_cdmid, how='left', on='cdmid')
# drop all columns but the entry identifier for the renewals
cdmid_matches = cdmid_matches[['uuid', 'renewal_uuid']]
cdmid_matches.head()

,uuid,renewal_uuid
0,01627bce16d2467cbac4756963a8d883,NaN
1,f0b04f5841ae41d39963289a7894ec9d,NaN
2,9aeff066dcf94164b08a40fca354fd23,NaN
3,1a91f4c0842140ca91ae89548c9588f6,NaN
4,b751d06548b64ddaa24e0ba239e3f4b7,NaN


In [0]:
# drop any non-matches
cdmid_matches.dropna(subset=['renewal_uuid'], inplace=True)
len(cdmid_matches)

136035

In [0]:
# any duplicate matches?
cdmid_matches['renewal_uuid'].duplicated().sum()

249

In [0]:
cdmid_matches.drop_duplicates(keep='first', inplace=True)
len(cdmid_matches)

136035

In [0]:
cdmid_matches.set_index('uuid', inplace=True)
cdmid_matches = cdmid_matches.groupby('uuid').agg(lambda x: '||'.join(x))
cdmid_matches.head(8)

,renewal_uuid
uuid,
0000216a4b144f6db2117feca322d10d,bb7e105e6a8845ad9ec46448cf0cbb12
00002a28abd54484a6218aa170c4926b,052e4e69a2ec4e59ad8e84e1abe950b8
00004e0edc9144df8826d74bc84dad2d,b3b8b0cec07143fdb506412b471f60ac
00006f1e1b7241209a9d1d17144c00da,c03fba8c417342aca9c6554c2f1a9a37
00008a3e335d4742a021bc31b4fc6df5,e484d9fb69ec474ab7f07ec6fd1b34f9
0001197b605f4f6fad856f89d716a882,82c5fcd351c44685a2d627bdd6c9fb1d
00014352923245039e30a090369489a7,14ef3d9be34a477191bdbef7c8faca72
00020e8019c44cbbbc97355cf8a05a37,0b19e007a2e64d16b7f7e4e4964b3a6c


In [0]:
len(cdmid_matches)

133661

## Pass Two

In [0]:
# now let's do it all over for cdmid_nd's; but first, we need to get rid of all of our
# cdmid matches in the renewals data
renewals_matched_by_cdmid = cdmid_matches['renewal_uuid'].to_list()
renewals_matched_by_cdmid_split = [x for i in renewals_matched_by_cdmid for x in i.split('||')]

In [0]:
len(renewals_matched_by_cdmid)

133661

In [0]:
len(renewals_matched_by_cdmid_split)

136035

In [0]:
deduped_renewals_matched_by_cdmid = list(set(renewals_matched_by_cdmid_split))
len(deduped_renewals_matched_by_cdmid)

135786

In [0]:
temp = pd.DataFrame(deduped_renewals_matched_by_cdmid)
temp['matched'] = 'yes'
temp.rename(columns={0: 'uuid'}, inplace=True)
temp.head()

,uuid,matched
0,fb4530f6a54947e2bd89e368072e01e3,yes
1,e998477206ea48b68d4cc4f39c285e7d,yes
2,c4892ba99c254cfe8d626e17a4a65957,yes
3,674e58464ff046039c12952a67239920,yes
4,017c54e6141f466ea7616460947c0ff6,yes


In [0]:
renewals_filtered = renewals_df_cleaned[['uuid', 'cdmid', 'cdmid_nd']]
len(renewals_filtered)

434174

In [0]:
matches = renewals_filtered.merge(temp, how='left', on='uuid')
len(matches)

434174

In [0]:
unmatched = matches[matches['matched'].isna()]
unmatched.head()

,uuid,cdmid,cdmid_nd,matched
0,03609b71da864fcc9a6319e58782cb1c,A778640|1924-03-28,A778640|1924,NaN
1,2af515ad21af405a94b45534ca94163a,B571853|1925-03-08,B571853|1925,NaN
2,ccc034a864fb40a68f099fafbbbddb96,A778640|1924-03-28,A778640|1924,NaN
3,2e0b51f7c4174e4db9c847386e3a0433,F42232|1925-04-10,F42232|1925,NaN
4,7f8d7aae0c504c41b27dc969a0cd4f2b,D64453|1925-05-12,D64453|1925,NaN


In [0]:
unmatched.drop(columns=['cdmid', 'matched'], inplace=True)
unmatched.rename(columns={"uuid": "renewal_uuid"}, inplace=True)
len(unmatched)

298388

In [0]:
unmatched.head()

,renewal_uuid,cdmid_nd
0,03609b71da864fcc9a6319e58782cb1c,A778640|1924
1,2af515ad21af405a94b45534ca94163a,B571853|1925
2,ccc034a864fb40a68f099fafbbbddb96,A778640|1924
3,2e0b51f7c4174e4db9c847386e3a0433,F42232|1925
4,7f8d7aae0c504c41b27dc969a0cd4f2b,D64453|1925


In [0]:
registrations_df_cnums_no_dates.reset_index(drop=False, inplace=True)
cdmid_nd_matches = registrations_df_cnums_no_dates.merge(unmatched, how='left', on='cdmid_nd')
# drop any non-matches
cdmid_nd_matches.dropna(subset=['renewal_uuid'], inplace=True)
len(cdmid_nd_matches)

316

In [0]:
# any duplicate matches?
cdmid_nd_matches['renewal_uuid'].duplicated().sum()

0

In [0]:
# drop all columns but the entry identifier for the renewals
cdmid_nd_matches = cdmid_nd_matches[['uuid', 'renewal_uuid']]
cdmid_nd_matches.set_index('uuid', inplace=True)
cdmid_nd_matches = cdmid_nd_matches.groupby('uuid').agg(lambda x: '||'.join(x))
cdmid_nd_matches.head(3)

,renewal_uuid
uuid,
0195b8865ba14b8d9faa730ad9bf4b2e,46eece3c1cca4a9b9f74e51c15210911
022f141f894344b7970752f6de8e463a,cb7218f018fc4dff99f301a717eb0372
0256f16258304b9d9dc633f56453e200,3cca639078e949c0bc40b95e2fe4c168


In [0]:
len(cdmid_nd_matches)

300

In [0]:
len(cdmid_matches)

133661

In [0]:
((len(cdmid_nd_matches) + len(cdmid_matches)) / len(registrations_df)) * 100

23.08450527048614

In [0]:
cdmid_nd_matches['match_method'] = 'cdmid_nd'
cdmid_matches['match_method'] = 'cdmid'

In [0]:
cdmid_nd_matches

,renewal_uuid,match_method
uuid,,
0195b8865ba14b8d9faa730ad9bf4b2e,46eece3c1cca4a9b9f74e51c15210911,cdmid_nd
022f141f894344b7970752f6de8e463a,cb7218f018fc4dff99f301a717eb0372,cdmid_nd
0256f16258304b9d9dc633f56453e200,3cca639078e949c0bc40b95e2fe4c168,cdmid_nd
051b328e522b45f5956507c6557256cc,94e71f6243474e559d7584494fc31023,cdmid_nd
0756c8a42eed466bb75614dc08512dcc,6ca7d4b18cbd4d9499563e5e957238d3,cdmid_nd
07a20138e3f64d5b8b5c1fcee4540c47,4c9b47e8214041eb8d7366cb8f0c250f,cdmid_nd
07f1637b70d14678a2caef161c52af92,bb53d522b9e34a27b9ea8704a552ff5c,cdmid_nd
08c768e78fe7415798220a0ea1e53f58,85dbde1b6bcb46a5b315db66e986a76e,cdmid_nd
08d7d2f4e75e4bf09e39f9098e940a31,426ce3b170db4e3fa77a11b5e08ac5b8,cdmid_nd


In [0]:
cdmid_matches

,renewal_uuid,match_method
uuid,,
0000216a4b144f6db2117feca322d10d,bb7e105e6a8845ad9ec46448cf0cbb12,cdmid
00002a28abd54484a6218aa170c4926b,052e4e69a2ec4e59ad8e84e1abe950b8,cdmid
00004e0edc9144df8826d74bc84dad2d,b3b8b0cec07143fdb506412b471f60ac,cdmid
00006f1e1b7241209a9d1d17144c00da,c03fba8c417342aca9c6554c2f1a9a37,cdmid
00008a3e335d4742a021bc31b4fc6df5,e484d9fb69ec474ab7f07ec6fd1b34f9,cdmid
0001197b605f4f6fad856f89d716a882,82c5fcd351c44685a2d627bdd6c9fb1d,cdmid
00014352923245039e30a090369489a7,14ef3d9be34a477191bdbef7c8faca72,cdmid
00020e8019c44cbbbc97355cf8a05a37,0b19e007a2e64d16b7f7e4e4964b3a6c,cdmid
000242dab93342e8b8a0ced16008f761,f79a3855c6a64fc985b77fcd15fb7a14,cdmid


In [0]:
all_matches = pd.concat([cdmid_matches, cdmid_nd_matches])

In [0]:
len(all_matches) / len(registrations_df) * 100

23.08450527048614

In [0]:
len(all_matches)

133961